#

In [1]:
import pandas as pd
import numpy as np

# 회전코드 사용시
import bottleneck as bn

# 경고문 무시
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# 시트 이름 확인
import openpyxl
wb = openpyxl.load_workbook(r'C:/Users/CVAI_EJ/Documents/반도체/6세대 스퍼터 데이터/RS Tune_v2.xlsx')
ws_naems = wb.sheetnames

for i in ws_naems:
    print(i)

V3 B #1-3ch (Ti)_60kW
V3 B #1-3ch (Ti)_50kW
V3 B #1-4ch (Al)_1000A_50KW
V3 B #1-4ch (Al)_3000A_50kW
V3 B #1-5ch (Al)_3000A_50kW
V3 B #1-5ch (Al)_3000A_60kW
V3 B #1-6ch (Ti)_500A_50kW
V3 B #1-6ch (Ti)_700A_50kW
C Ph2 A 3ch (Ti)
C Ph2 A 4ch (Al)
C Ph2 A 5ch (Al)_200kW
C Ph2 A 5ch (Al)_160kW
C Ph2 A 6ch (Ti)
B7 A #1-3ch (Ti)
B7 A #2-4ch (Al)_1600A
B7 A #2-4ch (Al)_3000A
B7 A #2-5ch (Al)_1600A
B7 A #2-5ch (Al)_3000A
B7 A #2-6ch (Ti)


In [3]:
# 90도 회전 
def rotated(array_2d):
    list_of_tuples = zip(*array_2d[::-1])
    return [list(elem) for elem in list_of_tuples]

# B7 함수 정의

In [13]:
# 엑셀에서 TM 부분만 데이터 가져오기
def B7_TM_2D(ws_lsit):
    df3 = pd.DataFrame()
    for ws in ws_lsit:
        df = pd.read_excel('C:/Users/CVAI_EJ/Documents/반도체/6세대 스퍼터 데이터/RS Tune_v2.xlsx', sheet_name=ws)
        index_num = df[df.iloc[:,2] ==735].index 
        df2 = pd.DataFrame()
        for i in index_num:
            df2 = df.iloc[i+1:i+7, 2:10]
            df3 = df3.append(df2, ignore_index=True)
    return df3

# 2D로 정리한 데이터 기판별로 저장
def B7_TM_array_1D(X): # X: 2D dataframe(B7_TM_2D 결과)
    N = int(X.shape[0]/6)
    
    # 2D로 정리한 데이터 기판별로 저장
    X_r = np.zeros((N, 6, 8))
    for k in range(N):
        for i in range(6):
            X_r[k, i, :] = X.iloc[k*6+i,:]
    

    # 홀수번 180회전 
    X_r2 = np.zeros((N, 6, 8))
    for i in range(N):
        if i%2==1 : #홀수 인 경우
            X_r2[i] = rotated(rotated(X_r[i])) # 90도 회전 두번 사용하여 180도 회전
        else:
            X_r2[i] = X_r[i] # 짝수인 경우는 그대로


    # 1차원으로 변환 후 데이터프레임으로 저장        
    X_re = np.zeros((N, 48))
    for k in range(N):
        X_re[k, :]  = np.reshape(X_r2[k],(48))
    df_X_re = pd.DataFrame(X_re.T)
    
    return X_r2, df_X_re

# 면저상 크기 5x8로 변환
def B7_TM_5x8_array_1D(X): # X: B7_TM_array_1D의 array
    N = len(X)
    
    # 이동평균으로 행1개 줄이기
    X_r3 = np.zeros((N, 5, 8))
    for i in range(N):
        a = bn.move_mean(X[i], window=2, min_count = None, axis=0) # 두개의 행을 평균내서 행 1개 줄이기
        a = a[~np.isnan(a)] # 결측값 삭제
        a = a.reshape(5,8) # 5x8로 reshape
        X_r3[i] = a
        
    # 1차원으로 변환 후 데이터프레임으로 저장 
    X_re = np.zeros((N, 40))
    for k in range(N):
        X_re[k, :]  = np.reshape(X_r3[k],(40))
    df_X_re = pd.DataFrame(X_re.T)
    
    return X_r3, df_X_re

# 면저상 크기 4x9로 변환
def B7_TM_4x9(X):  # X: B7_TM_5x8_array_1D의 array
    N = len(X)
    
     # 이동평균으로 행1개 줄이기
    X_r4 = np.zeros((N, 4, 8))
    for i in range(N):
        a = bn.move_mean(X[i], window=2, min_count = None, axis=0)
        a = a[~np.isnan(a)]
        a = a.reshape(4,8)
        X_r4[i] = a
    
    # 열 1개 늘리기
    X_r5 = np.zeros((N, 4, 9))
    for n in range(N):
        X_r5[n, : , 0] = X_r4[n, : , 0]
        X_r5[n, : , 1] = X_r4[n, : , 1]
        X_r5[n, : , 2] = X_r4[n, : , 2]
        X_r5[n, : , 3] = X_r4[n, : , 3]
        X_r5[n, : , 4] = ((X_r4[n,:,3])*0.5) + ((X_r4[n,:,4])*0.5)
        X_r5[n, : , 5] = X_r4[n, : , 4]
        X_r5[n, : , 6] = X_r4[n, : , 5]
        X_r5[n, : , 7] = X_r4[n, : , 6]
        X_r5[n, : , 8] = X_r4[n, : , 7]
    
    # 1차원으로 변환 후 데이터프레임으로 저장
    X_re = np.zeros((N, 36))
    for k in range(N):
        X_re[k, :]  = np.reshape(X_r5[k],(36))
    df_X_re = pd.DataFrame(X_re.T)
    
    return df_X_re

def B7_MG_2D(ws_lsit):
    # 마그넷 시작열이 E열에서 시작
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    for ws in ws_lsit:
        df = pd.read_excel('C:/Users/CVAI_EJ/Documents/반도체/6세대 스퍼터 데이터/RS Tune_v2.xlsx', sheet_name=ws)
        index_num = df[df.iloc[:,5] =='MG1'].index
        for i in index_num:
            df2 = df.iloc[i+1:i+5, 5:14]
            df3 = df3.append(df2, ignore_index=True)
            df3 = df3.append(df2, ignore_index=True)
    return df3

def B7_MG_1D(Y):
    N = int(Y.shape[0]/4)
    Y_r = np.zeros((N, 4, 9))
    for k in range(N):
        for i in range(4):
            Y_r[k, i, :] = Y.iloc[k*4+i,:]
    Y_re = np.zeros((N, 36))
    for k in range(N):
        Y_re[k, :]  = np.reshape(Y_r[k],(36))        
    df_Y_re = pd.DataFrame(Y_re.T)
    return df_Y_re

# B7

In [9]:
ws_lsit = [
'B7 A #1-3ch (Ti)',
'B7 A #2-4ch (Al)_1600A',
'B7 A #2-4ch (Al)_3000A',
'B7 A #2-5ch (Al)_1600A',
'B7 A #2-5ch (Al)_3000A',
'B7 A #2-6ch (Ti)'
]

B7_TM_array, B7_TM  = B7_TM_array_1D(B7_TM_2D(ws_lsit))
B7_TM_5x8_array, B7_TM_5x8 = B7_TM_5x8_array_1D(B7_TM_array)
B7_TM_4x9 = B7_TM_4x9(B7_TM_5x8_array)
B7_MG = B7_MG_1D(B7_MG_2D(ws_lsit))

B7_TM.to_csv('../data/B7/B7_TM.csv',index=False)
B7_TM_5x8.to_csv('../data/B7/B7_TM_5x8.csv',index=False)
B7_TM_4x9.to_csv('../data/B7/B7_TM_4x9.csv',index=False)
B7_MG.to_csv('../data/B7/B7_MG.csv',index=False)

In [12]:
ws_lsit = [
'B7 A #2-4ch (Al)_1600A',
'B7 A #2-4ch (Al)_3000A',
'B7 A #2-5ch (Al)_1600A',
'B7 A #2-5ch (Al)_3000A',
]

B7_TM_array, B7_TM  = B7_TM_array_1D(B7_TM_2D(ws_lsit))
B7_TM_5x8_array, B7_TM_5x8 = B7_TM_5x8_array_1D(B7_TM_array)
B7_TM_4x9 = B7_TM_4x9(B7_TM_5x8_array)
B7_MG = B7_MG_1D(B7_MG_2D(ws_lsit))

B7_TM.to_csv('../data/B7/B7_Al_TM.csv',index=False)
B7_TM_5x8.to_csv('../data/B7/B7_Al_TM_5x8.csv',index=False)
B7_TM_4x9.to_csv('../data/B7/B7_Al_TM_4x9.csv',index=False)
B7_MG.to_csv('../data/B7/B7_Al_MG.csv',index=False)

In [14]:
ws_lsit = [
'B7 A #1-3ch (Ti)',
'B7 A #2-6ch (Ti)'
]

B7_Ti_TM_array, B7_Ti_TM  = B7_TM_array_1D(B7_TM_2D(ws_lsit))
B7_Ti_TM_5x8_array, B7_Ti_TM_5x8 = B7_TM_5x8_array_1D(B7_Ti_TM_array)
B7_Ti_TM_4x9 = B7_TM_4x9(B7_Ti_TM_5x8_array) 
B7_Ti_MG = B7_MG_1D(B7_MG_2D(ws_lsit))

B7_Ti_TM.to_csv('../data/B7/B7_Ti_TM.csv',index=False)
B7_Ti_TM_5x8.to_csv('../data/B7/B7_Ti_TM_5x8.csv',index=False)
B7_Ti_TM_4x9.to_csv('../data/B7/B7_Ti_TM_4x9.csv',index=False)
B7_Ti_MG.to_csv('../data/B7/B7_Ti_MG.csv',index=False)


# V3 함수 정의

In [20]:
def V3_TM_2D(ws_lsit):
    df3 = pd.DataFrame()
    for ws in ws_lsit:
        df = pd.read_excel('C:/Users/CVAI_EJ/Documents/반도체/6세대 스퍼터 데이터/RS Tune_v2.xlsx', sheet_name=ws)
        index_num = df[df.iloc[:,2] ==735].index 
        df2 = pd.DataFrame()
        for i in index_num:
            df2 = df.iloc[i+1:i+6, 2:11]
            df3 = df3.append(df2, ignore_index=True)
    return df3

def V3_TM_array_1D(X): # X: 2D dataframe(V3_TM_2D 결과)
    N = int(X.shape[0]/5)
    
    # 2D로 정리한 데이터 기판별로 저장
    X_r = np.zeros((N,5,9))
    for k in range(N):
        for i in range(5):
            X_r[k, i, :] = X.iloc[k*5+i,:]
    

    # 홀수번 180회전 
    X_r2 = np.zeros((N,5,9))
    for i in range(N):
        if i%2==1 : #홀수 인 경우
            X_r2[i] = rotated(rotated(X_r[i])) # 90도 회전 두번 사용하여 180도 회전
        else:
            X_r2[i] = X_r[i] # 짝수인 경우는 그대로


    # 1차원으로 변환 후 데이터프레임으로 저장        
    X_re = np.zeros((N, 45))
    for k in range(N):
        X_re[k, :]  = np.reshape(X_r2[k],(45))
    df_X_re = pd.DataFrame(X_re.T)
    
    return X_r2, df_X_re

def V3_TM_5x8_array_1D(X): # X: V3_TM_array_1D의 array
    N = len(X)
    
    # 이동평균으로 행1개 줄이기
    X_r3 = np.zeros((N, 5, 8))
    for i in range(N):
        a = bn.move_mean(X[i], window=2, min_count = None, axis=1)
        a  =np.delete(a, 0 , axis = 1)
        a = a.reshape(5,8)
        X_r3[i] = a
        
    # 1차원으로 변환 후 데이터프레임으로 저장 
    X_re = np.zeros((N, 40))
    for k in range(N):
        X_re[k, :]  = np.reshape(X_r3[k],(40))
    df_X_re = pd.DataFrame(X_re.T)
    
    return df_X_re

def V3_TM_4x9(X):  # X: V3_TM_5x8_array_1D의 array
    N = len(X)
    
     # 이동평균으로 행1개 줄이기
    X_r4 = np.zeros((N, 4, 9))
    for i in range(N):
        a = bn.move_mean(X[i], window=2, min_count = None, axis=0)
        a = np.delete(a, 0 , axis = 0)
        a = a.reshape(4,9)
        X_r4[i] = a

    # 1차원으로 변환 후 데이터프레임으로 저장
    X_re = np.zeros((N, 36))
    for k in range(N):
        X_re[k, :]  = np.reshape(X_r4[k],(36))
    df_X_re = pd.DataFrame(X_re.T)
    
    return df_X_re

def V3_MG_2D(ws_lsit):
    # 마그넷 시작열이 E열에서 시작
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    for ws in ws_lsit:
        df = pd.read_excel('C:/Users/CVAI_EJ/Documents/반도체/6세대 스퍼터 데이터/RS Tune_v2.xlsx', sheet_name=ws)
        index_num = df[df.iloc[:,4] =='MG1'].index
        for i in index_num:
            df2 = df.iloc[i+1:i+5, 4:13]
            df3 = df3.append(df2, ignore_index=True)
            df3 = df3.append(df2, ignore_index=True)

    return df3

def V3_MG_1D(Y):
    N = int(Y.shape[0]/4)
    Y_r = np.zeros((N, 4, 9))
    for k in range(N):
        for i in range(4):
            Y_r[k, i, :] = Y.iloc[k*4+i,:]
    Y_re = np.zeros((N, 36))
    for k in range(N):
        Y_re[k, :]  = np.reshape(Y_r[k],(36))        
    df_Y_re = pd.DataFrame(Y_re.T)
    return df_Y_re

In [16]:
# ALL(TI+Al) data
ws_lsit = [
'V3 B #1-3ch (Ti)_60kW',
'V3 B #1-3ch (Ti)_50kW',
'V3 B #1-4ch (Al)_1000A_50KW',
'V3 B #1-4ch (Al)_3000A_50kW',
'V3 B #1-5ch (Al)_3000A_50kW',
'V3 B #1-5ch (Al)_3000A_60kW',
'V3 B #1-6ch (Ti)_500A_50kW',
'V3 B #1-6ch (Ti)_700A_50kW',
]

V3_TM_array, V3_TM  = V3_TM_array_1D(V3_TM_2D(ws_lsit))
V3_TM_5x8 = V3_TM_5x8_array_1D(V3_TM_array)
V3_TM_4x9 = V3_TM_4x9(V3_TM_array)
V3_MG = V3_MG_1D(V3_MG_2D(ws_lsit))

V3_TM.to_csv('../data/V3/V3_TM.csv',index=False)
V3_TM_5x8.to_csv('../data/V3/V3_TM_5x8.csv',index=False)
V3_TM_4x9.to_csv('../data/V3/V3_TM_4x9.csv',index=False)
V3_MG.to_csv('../data/V3/V3_MG.csv',index=False)

In [19]:
# TI data
ws_lsit = [
'V3 B #1-3ch (Ti)_60kW',
'V3 B #1-3ch (Ti)_50kW',
'V3 B #1-6ch (Ti)_500A_50kW',
'V3 B #1-6ch (Ti)_700A_50kW',
]

V3_Ti_TM_array, V3_Ti_TM  = V3_TM_array_1D(V3_TM_2D(ws_lsit))
V3_Ti_TM_5x8 = V3_TM_5x8_array_1D(V3_Ti_TM_array)
V3_Ti_TM_4x9 = V3_TM_4x9(V3_Ti_TM_array)
V3_Ti_MG = V3_MG_1D(V3_MG_2D(ws_lsit))

V3_Ti_TM.to_csv('../data/V3/V3_Ti_TM.csv',index=False)
V3_Ti_TM_5x8.to_csv('../data/V3/V3_Ti_TM_5x8.csv',index=False)
V3_Ti_TM_4x9.to_csv('../data/V3/V3_Ti_TM_4x9.csv',index=False)
V3_Ti_MG.to_csv('../data/V3/V3_Ti_MG.csv',index=False)

In [21]:
# Al data
ws_lsit = [
'V3 B #1-4ch (Al)_1000A_50KW',
'V3 B #1-4ch (Al)_3000A_50kW',
'V3 B #1-5ch (Al)_3000A_50kW',
'V3 B #1-5ch (Al)_3000A_60kW',
]

V3_Al_TM_array, V3_Al_TM  = V3_TM_array_1D(V3_TM_2D(ws_lsit))
V3_Al_TM_5x8 = V3_TM_5x8_array_1D(V3_Al_TM_array)
V3_Al_TM_4x9 = V3_TM_4x9(V3_Al_TM_array)
V3_Al_MG = V3_MG_1D(V3_MG_2D(ws_lsit))

V3_Al_TM.to_csv('../data/V3/V3_Al_TM.csv',index=False)
V3_Al_TM_5x8.to_csv('../data/V3/V3_Al_TM_5x8.csv',index=False)
V3_Al_TM_4x9.to_csv('../data/V3/V3_Al_TM_4x9.csv',index=False)
V3_Al_MG.to_csv('../data/V3/V3_Al_MG.csv',index=False)